<a href="https://colab.research.google.com/github/mmarushika/sdc-lab/blob/main/RNNGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install Gradio
!pip install -q gradio

# Step 2: Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import gradio as gr

# Step 3: Load and preprocess data
vocab_size = 10000
maxlen = 200

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Step 4: Build and train the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=32, input_length=maxlen),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model (just 2 epochs for speed — increase for better performance)
model.fit(x_train, y_train, epochs=2, batch_size=64, validation_split=0.2)

# Step 5: Prepare word index for decoding
word_index = imdb.get_word_index()
index_word = {v + 3: k for k, v in word_index.items()}
index_word[0] = "<PAD>"
index_word[1] = "<START>"
index_word[2] = "<UNK>"

# Step 6: Function to encode text input
def encode_review(text):
    words = text.lower().split()
    encoded = [1]  # Start token
    for word in words:
        idx = word_index.get(word, 2)  # 2 is <UNK>
        if idx < vocab_size:
            encoded.append(idx)
    return pad_sequences([encoded], maxlen=maxlen)

# Step 7: Gradio prediction function
def predict_sentiment(text):
    encoded = encode_review(text)
    prediction = model.predict(encoded)[0][0]
    label = "Positive 😀" if prediction >= 0.5 else "Negative 😞"
    return f"{label} (Confidence: {prediction:.2f})"

# Step 8: Launch Gradio interface
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=4, placeholder="Enter a movie review here..."),
    outputs="text",
    title="🎬 Movie Review Sentiment Analyzer (RNN)",
    description="Enter a movie review and see if the sentiment is positive or negative using an RNN model."
)

interface.launch(share=True)  # share=True to get a public link in Colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 70ms/step - accuracy: 0.5796 - loss: 0.6603 - val_accuracy: 0.7462 - val_loss: 0.5163
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 68ms/step - accuracy: 0.8120 - loss: 0.4172 - val_accuracy: 0.8368 - val_loss: 0.3879
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8641c442c12c2f21d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
